In [ ]:
# This generates configuration values with Capytaine.

#!pip install capytaine #uncomment if first time running

import capytaine as cpt
import numpy as np
import matplotlib.pyplot as plt
import copy
from capytaine.bem.airy_waves import airy_waves_potential, airy_waves_velocity, froude_krylov_force


In [ ]:
def save_potential_array(title, arr):
    file_path = title + "-real" + ".csv"
    np.savetxt(file_path, np.real(arr), delimiter=",", fmt="%.6e")
    file_path = title + "-imag" + ".csv"
    np.savetxt(file_path, np.imag(arr), delimiter=",", fmt="%.6e")

In [ ]:
def body_from_profile(x,y,z,nphi):
    xyz = np.array([np.array([x/np.sqrt(2),y/np.sqrt(2),z]) for x,y,z in zip(x,y,z)])    # /sqrt(2) to account for the scaling
    body = cpt.FloatingBody(cpt.AxialSymmetricMesh.from_profile(xyz, nphi=nphi))
    return body

def make_face(p1, p2, f_density, t_density):
    zarr = np.linspace(p1[1], p2[1], f_density)
    rarr = np.linspace(p1[0], p2[0], f_density)
    return body_from_profile(rarr, rarr, zarr, t_density)

def get_points(a, d): # These points define the outline of the body
    d_prime = d + [0]
    d_index = 0
    a_index = 0
    pt_lst = [(0, - d[0])]
    for i in range(len(a)):
        pt_lst.append((a[a_index], - d_prime[d_index]))
        d_index +=1
        pt_lst.append((a[a_index], - d_prime[d_index]))
        a_index+=1
    return pt_lst

# compute number of panels along each surface given total number along the outline
def get_f_densities(pt_lst, total_units):
    face_lengths = np.array([])
    for i in range(len(pt_lst) - 1):
        p1, p2 = pt_lst[i], pt_lst[i + 1]
        face_length = abs(p2[0] - p1[0]) + abs(p2[1] - p1[1]) # one of these two values will be zero
        face_lengths = np.append(face_lengths, face_length)
    total_length = sum(face_lengths)
    each_face_densities = np.vectorize(lambda x: max(1, x/total_length * total_units))(face_lengths) # each face needs at least one panel
    remainders = each_face_densities % 1
    each_face_densities = each_face_densities.astype(int)
    remaining_units = total_units - sum(each_face_densities)
    if remaining_units < 0: # high proportion of small faces
        for u in range(remaining_units * -1):
            i = np.argmax(each_face_densities) # cut density from the largest faces
            each_face_densities[i] = (each_face_densities[i]) - 1
    else:
        for u in range(remaining_units): # distribute remaining units where most needed
            i = np.argmax(remainders)
            each_face_densities[i] = (each_face_densities[i]) + 1
            remainders[i] = 0
    assert sum(each_face_densities) == total_units
    return each_face_densities
    
def make_body(a, d, heaving, t_densities, face_units): 
    pts = get_points(a,d)
    f_densities = get_f_densities(pts, face_units)
    faces_and_heaves = []
    panel_ct = 0
    for i in range((len(pts) - 1) // 2):
        p1, p2, p3 = pts[2 * i], pts[2 * i + 1], pts[2 * i + 2]
        # make a horizontal face
        h_face = make_face(p1, p2, f_densities[2 * i], t_densities[i])
        h_heave = heaving[i]
        faces_and_heaves.append((h_face, h_heave))
        panel_ct += f_densities[2 * i] * t_densities[i]
        # make a vertical face
        if p2[1] < p3[1]: # body on left
            region = i
        else: # body on right
            region = i + 1
        v_face = make_face(p2, p3, f_densities[2 * i + 1], t_densities[region])
        faces_and_heaves.append((v_face, heaving[region]))
        panel_ct += f_densities[2 * i + 1] * t_densities[region]
    body = add_heaves(faces_and_heaves)
    return body, panel_ct
    

def add_heaves(faces_and_heaves):
    hcreate = False
    screate = False
    for fh in faces_and_heaves: # Splits list of faces into those that are heaving and those that are not.
        if fh[1]: #fh of the form (face, heaving)
            if not hcreate:
                heaving_body = fh[0]
                hcreate = True
            else:
                heaving_body = heaving_body + fh[0]
        else:
            if not screate:
                still_body = fh[0]
                screate = True
            else:
                still_body = still_body + fh[0]
    if hcreate: # Adds heave dof to the heaving collection
        heaving_body.add_translation_dof(name='Heave')
        if screate:
            return (heaving_body + still_body)
        else:
            return (heaving_body)
    else:
        return (still_body)

###################################
# Solving
solver = cpt.BEMSolver()

def rb_solve(a, d, heaving, t_densities, face_units, m0, h, rho):
    body, panel_count = make_body(a, d, heaving, t_densities, face_units)
    body = body.immersed_part() # removes points above z = 0
    body.show_matplotlib()
    
    rad_problem = cpt.RadiationProblem(body = body, wavenumber = m0, water_depth = h, rho = rho)
    results = solver.solve(rad_problem, keep_details = True)
    print(results.added_mass)
    print(results.radiation_damping)
    print("# of Panels: ", panel_count)
    return results, panel_count

In [ ]:
# original - two cylinder
h = 1.001
d = [0.5, 0.25]
a = [0.5, 1]
heaving = [1, 1]
t_densities = [50, 100] # number of panels around each cylinder
face_units = 90 # number of panels along the outline of the configuration
m0 = 1
rho = 1023 # density of our special material
config = "config0"

result, panel_count = rb_solve(a, d, heaving, t_densities, face_units, m0, h, rho)

In [ ]:
#staircase  - compound cylinder
h = 1.5
d = [1.1, 0.85, 0.75, 0.4, 0.15]
a = [0.3, 0.5, 1, 1.2, 1.6]
heaving = [1, 1, 1, 1, 1]
t_densities = [30, 50, 100, 120, 160]
face_units = 93
m0 = 1
rho = 1023
config = "config1"


result, panel_count = rb_solve(a, d, heaving, t_densities, face_units, m0, h, rho)

In [ ]:
#tall  - compound cylinder
h = 100
d = [29, 7, 4]
a = [3, 5, 10]
heaving = [1, 1, 1]
t_densities = [30, 50, 100]
face_units = 110
m0 = 1
rho = 1023
config = "config2"

result, panel_count = rb_solve(a, d, heaving, t_densities, face_units, m0, h, rho)

In [ ]:
#indents  - compound cylinder
h = 1.9
d = [0.5, 0.7, 0.8, 0.2, 0.5]
a = [0.3, 0.5, 1, 1.2, 1.6]
heaving = [1, 1, 1, 1, 1]
t_densities = [30, 50, 100, 120, 160]
face_units = 105
m0 = 1
rho = 1023
config = "config3"

result, panel_count = rb_solve(a, d, heaving, t_densities, face_units, m0, h, rho)

In [ ]:
#original - only outer heaving
h = 1.001
d = [0.5, 0.25]
a = [0.5, 1]
heaving = [0, 1]
t_densities = [50, 100]
face_units = 90
m0 = 1
rho = 1023
config = "config4"

result, panel_count = rb_solve(a, d, heaving, t_densities, face_units, m0, h, rho)

In [ ]:
#original - only inner heaving
h = 1.001
d = [0.5, 0.25]
a = [0.5, 1]
heaving = [1, 0]
t_densities = [50, 100]
face_units = 90
m0 = 1
rho = 1023
config = "config5"

result, panel_count = rb_solve(a, d, heaving, t_densities, face_units, m0, h, rho)

In [ ]:
#tall  - spar not heaving
h = 100
d = [29, 7, 4]
a = [3, 5, 10]
heaving = [0, 1, 1]
t_densities = [30, 50, 100]
face_units = 110
m0 = 1
rho = 1023
config = "config6"

result, panel_count = rb_solve(a, d, heaving, t_densities, face_units, m0, h, rho)

In [ ]:
# Get potentials
# Define the ranges for R and Z
R_range = np.linspace(0.0, 2*a[-1], num=50)
theta_range = np.linspace(-np.pi, np.pi, num=4)
Z_range = np.linspace(0, -h, num=50) #h

# Create mesh grids for R, theta, and Z
R, theta, Z = np.meshgrid(R_range, theta_range, Z_range, indexing='ij')

# Convert cylindrical coordinates to Cartesian coordinates for capytaine
X = R * np.cos(theta)
Y = R * np.sin(theta)
Z = Z
# Create an array of shape (N, 3)
points = np.zeros((R.size, 3))

# Assign the values of R, Z, and y to the array
points[:, 0] = X.ravel()
points[:, 1] = Y.ravel()
points[:, 2] = Z.ravel()
#need cartesian here
phi_inc = solver.compute_potential(points,result) #rad problem

regions = []
regions.append((R <= a[0]) & (Z > -d[0]))
for i in range(1, len(a)):
    regions.append((R > a[i-1]) & (R <= a[i]) & (Z > -d[i]))
regions.append(R > a[-1])

# Apply masks to create a blank plot in specified regions
phi_inc = phi_inc.reshape((50,4,50))

for i in range(len(a)):
  phi_inc[regions[i]] = np.nan



In [ ]:
# Get velocities
vel_inc = solver.compute_velocity(points,result)
velx_inc = vel_inc[:,0].reshape((50,4,50))
vely_inc = vel_inc[:,1].reshape((50,4,50))
velz_inc = vel_inc[:,2].reshape((50,4,50))
for i in range(len(a)):
  velx_inc[regions[i]] = np.nan
  vely_inc[regions[i]] = np.nan
  velz_inc[regions[i]] = np.nan

In [ ]:
# Plot potentials and velocities
# The slicing limits the y-value to 0 because we only care about the x-z (r-z) plane.
plt.contourf(R[:, 0, :], Z[:, 0, :], phi_inc[:, 0, :], cmap='viridis', levels = 50)
plt.colorbar(label='Potential')
plt.contour(R[:, 0, :], Z[:, 0, :], phi_inc[:, 0, :], colors='black', linestyles='solid', linewidths=0.05,levels=50)

# Add labels and title
plt.xlabel('R')
plt.ylabel('Z')
plt.title('Contour Plot of Re(Potential) using BEM')

plt.show()

imag_phi_inc = np.imag(phi_inc[:, 0, :])

nan_mask = np.isnan(np.real(phi_inc[:, 0, :]))

np.imag(phi_inc[:, 0, :])[nan_mask] = np.nan

plt.contourf(R[:, 0, :], Z[:, 0, :], imag_phi_inc, cmap='viridis', levels = 50)
plt.colorbar(label='Potential')
plt.contour(R[:, 0, :], Z[:, 0, :], imag_phi_inc, colors='black', linestyles='solid', linewidths=0.05,levels=50)


# Add labels and title
plt.xlabel('R')
plt.ylabel('Z')
plt.title('Contour Plot of Im(Potential) using BEM')

plt.show()

def plot_vel(data, title):
  plt.contourf(R[:, 0, :], Z[:, 0, :], data[:, 0, :], cmap='viridis', levels = 50)
  plt.colorbar(label='V')
  plt.contour(R[:, 0, :], Z[:, 0, :], data[:, 0, :], colors='black', linestyles='solid', linewidths=0.05,levels=50)

  # Add labels and title
  plt.xlabel('R')
  plt.ylabel('Z')
  plt.title(title)

  plt.show()

nan_mask = np.isnan(np.real(velx_inc))

velx_imag = np.imag(velx_inc)
velz_imag = np.imag(velz_inc)

velx_imag[nan_mask] = np.nan
velz_imag[nan_mask] = np.nan

plot_vel(velx_inc, "Contour Plot of Re(Vx) using BEM")
plot_vel(velx_imag, "Contour Plot of Im(Vx) using BEM")
plot_vel(velz_inc, "Contour Plot of Re(Vz) using BEM")
plot_vel(velz_imag, "Contour Plot of Im(Vz) using BEM")


In [ ]:
save_potential_array(config, phi_inc[:, 0, :])
# WARNING: This overwrites existing files with the same name. Ensure that <config> is correct before running.